## Repartição e processamento de arquivo
Esse script tem o objetivo de extrair as colunas desejadas de um csv grande e separar o resultado em três arquivos menores.

### Import do pandas

In [ ]:
import pandas as pd

### Carregamento do Dicionário de Dados e Extração de Colunas

1. **Arquivo de Dados**: `arquivo.csv` será processado posteriormente.
2. **Dicionário de Dados**: A célula carrega o arquivo `dicionario.xlsx`, que contém o dicionário de colunas que serão utilizadas no processamento.
3. **Extração de Colunas**: Um loop percorre as linhas do dicionário de dados para armazenar o nome de cada coluna em uma lista chamada `columns_dict`.

In [ ]:
file = "arquivo.csv"
df_dict = pd.read_excel("dicionario.xlsx")
columns_dict = []

for campo in df_dict.values:
    columns_dict.append(campo[0])

### Visualizando e removendo as colunas indesejáveis

Além da extração das colunas corretas, vieram 3 colunas que não serão utilizadas na tabela final.

In [ ]:
columns_dict[:3]

[nan, nan, 'Campo']

In [ ]:
columns_dict.pop(0)
columns_dict.pop(0)
columns_dict.pop(0)

'Campo'

### Ajuste de Nomes de Colunas no Dicionário de Dados

Este bloco de código:
- Substitui o nome da coluna `cd_ibge` por `d.cd_ibge`.
- Substitui o nome da coluna `p.id_familia` por `d.id_familia`.

In [ ]:
indice = columns_dict.index("cd_ibge")
columns_dict[indice] = "d.cd_ibge"

indice = columns_dict.index("p.id_familia")
columns_dict[indice] = "d.id_familia"

### Filtragem e Exportação de Dados Processados

- O arquivo original é carregado como `raw_df`, com todas as colunas convertidas para `str` e o separador `;`.
- Um novo CSV (`table_processed.csv`) é gerado, incluindo apenas as colunas especificadas em `columns_dict`, e sem o índice.

In [ ]:
raw_df = pd.read_csv(file, sep=';', dtype=str)
raw_df.to_csv("table_processed.csv", columns=columns_dict, index=False )

### Carregamento dos Dados Processados

- Carrega o arquivo CSV `table_processed.csv` (gerado na etapa anterior) no DataFrame `processed_df`, com todas as colunas convertidas para o tipo `str`.
- Esse DataFrame agora contém apenas as colunas selecionadas e padronizadas do arquivo original. 
- Entretanto, o arquivo em questão ainda possui muitas linhas, impedindo sua total visualização em editores convencionias de csv.
- Portanto, é necessária uma próxima etapa para particionar esse arquivo muito grande em arquivos menores.


In [ ]:
processed_df = pd.read_csv("table_processed.csv", dtype=str)

### Agrupando o total de linhas de cada cidade

- Cria-se um objeto Series contendo o total de linhas no csv agrupadas pelo `d.cd_ibge` (código do ibge de cada cidade) e ordenados de forma descendente.

In [ ]:
qty_lines_groupedby_cities = processed_df.groupby(['d.cd_ibge'])['d.cd_ibge'].count().sort_values(ascending=False)

- Definindo quantas cidades irão para cada parte do arquivo baseado nas ocorrências de seus `d.cd_ibge`.

In [ ]:
sum(qty_lines_groupedby_cities[:7]) #808593
sum(qty_lines_groupedby_cities[7:60]) #870543
sum(qty_lines_groupedby_cities[60:]) #788609

808593 + 870543 + 788609 

2467745

### Divisão do Dataset em Arquivos CSV por Faixa de Cidades

Este bloco de código divide o dataset em diferentes partes, criando arquivos CSV distintos :

- **Agrupamento de Códigos IBGE**: Coleta os códigos IBGE em três faixas de índices (primeiros 7, entre 7 e 60, e acima de 60).
- **Criação de Arquivo CSV**: A cada ponto de corte (`6`, `59` ou final da lista), filtra o DataFrame `processed_df` para as cidades agrupadas, salva o resultado em um arquivo CSV e prepara para o próximo grupo.
- **Controle de Partes**: `parte` é incrementado para garantir que os arquivos CSV sejam nomeados sequencialmente.


In [ ]:
codigo_ibge_cidades = []
parte = 1

for idx, _ in enumerate(qty_lines_groupedby_cities):
    if idx < 7:
        codigo_ibge_cidades.append(qty_lines_groupedby_cities.index[idx])
    elif 7 <= idx < 60:
        codigo_ibge_cidades.append(qty_lines_groupedby_cities.index[idx])
    elif 60 <= idx:
        codigo_ibge_cidades.append(qty_lines_groupedby_cities.index[idx])

    if idx in [6, 59, len(qty_lines_groupedby_cities) - 1]:
        
        final_csv_lines = []
        
        for codigo_ibge in codigo_ibge_cidades:
            linha = processed_df[processed_df['d.cd_ibge'] == codigo_ibge].values
            final_csv_lines.extend(list(linha))
        table_cad_part = pd.DataFrame(columns=processed_df.columns, data=final_csv_lines)
        table_cad_part.to_csv(f"table_part{parte}.csv", index=False)

        codigo_ibge_cidades.clear()
        parte +=1

In [ ]:
df1 = pd.read_csv("table_part1.csv", dtype=str)
df2 = pd.read_csv("table_part2.csv", dtype=str)
df3 = pd.read_csv("table_part3.csv", dtype=str)



In [ ]:
len(df1) + len(df2) + len(df3) == len(processed_df)


True